# Hunting With Data Science

### Parte 1 - Filtrado y preprocesamiento

In [16]:
import pandas as pd
from pandas import json_normalize

In [10]:
df = pd.read_json("Files/large_eve.json", lines=True)
print(df.info)

<bound method DataFrame.info of                               timestamp       flow_id  pcap_cnt event_type  \
0       2017-07-22T17:33:15.784100-0500  2.835707e+14   18519.0      alert   
1       2017-07-22T17:33:16.571273-0500  1.519048e+15   21531.0        ssh   
2       2017-07-22T17:33:16.661646-0500  1.327836e+15   22269.0        dns   
3       2017-07-22T17:33:17.561063-0500  9.632021e+14   28731.0       http   
4       2017-07-22T17:33:17.595416-0500  9.632021e+14   28994.0   fileinfo   
...                                 ...           ...       ...        ...   
746904  2017-07-22T19:47:49.301548-0500  1.970284e+15       NaN       flow   
746905  2017-07-22T19:47:49.301548-0500  2.251792e+15       NaN       flow   
746906  2017-07-22T19:47:49.301548-0500  8.444188e+14       NaN       flow   
746907  2017-07-22T19:47:49.301548-0500  2.814688e+14       NaN       flow   
746908  2017-07-19T17:39:33.004305-0400           NaN       NaN      stats   

         vlan                  

In [22]:
df_dns = df[df['event_type'] == 'dns']
df_dns.info

<bound method DataFrame.info of                               timestamp       flow_id   pcap_cnt event_type  \
2       2017-07-22T17:33:16.661646-0500  1.327836e+15    22269.0        dns   
38      2017-07-22T17:33:24.990320-0500  2.022925e+15    54352.0        dns   
39      2017-07-22T17:33:27.379891-0500  5.785448e+14    54519.0        dns   
40      2017-07-22T17:33:27.380146-0500  5.785448e+14    54520.0        dns   
41      2017-07-22T17:33:27.380146-0500  5.785448e+14    54520.0        dns   
...                                 ...           ...        ...        ...   
716752  2017-07-22T19:47:03.373435-0500  8.918761e+14  4441474.0        dns   
716753  2017-07-22T19:47:03.373435-0500  8.918761e+14  4441474.0        dns   
716893  2017-07-22T19:47:38.146503-0500  2.199316e+15  4452917.0        dns   
716894  2017-07-22T19:47:38.146953-0500  2.199316e+15  4452918.0        dns   
716895  2017-07-22T19:47:38.146953-0500  2.199316e+15  4452918.0        dns   

         vlan      

In [23]:
df_dns.head(2)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df['dns'] = df['dns'].apply(lambda x: [x] if isinstance(x, dict) else x)
df_normalized = pd.json_normalize(df.to_dict('records'), 'dns', ['timestamp', 'flow_id', 'pcap_cnt', 'event_type', 'vlan', 'src_ip', 'src_port', 'dest_ip', 'dest_port', 'proto'])
df_normalized.head()

,type,id,rrname,rrtype,tx_id,rcode,ttl,rdata,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto
0,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:16.661646-0500,1327836194150542.0,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP
1,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:24.990320-0500,2022925111925872.0,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP
2,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:27.379891-0500,578544790391795.0,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP
3,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN,2017-07-22T17:33:27.380146-0500,578544790391795.0,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP
4,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN,2017-07-22T17:33:27.380146-0500,578544790391795.0,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP


In [29]:
df_dns_a = df_normalized[df_normalized['rrtype'] == 'A']
print(df_dns_a.shape)
df_dns_a.head()

(2849, 18)


,type,id,rrname,rrtype,tx_id,rcode,ttl,rdata,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto
0,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:16.661646-0500,1327836194150542.0,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP
1,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:24.990320-0500,2022925111925872.0,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP
2,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:27.379891-0500,578544790391795.0,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP
5,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:36.672785-0500,237919524635665.0,55496.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,41663.0,2001:07fd:0000:0000:0000:0000:0000:0001,53.0,UDP
6,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,2017-07-22T17:33:38.537426-0500,2167545251640146.0,55687.0,dns,180.0,192.168.198.62,35092.0,192.168.207.4,53.0,UDP


In [37]:
dominios_unicos = df_dns_a['rrname'].unique()
print("Hay " + str(len(dominios_unicos)) + " registros unicos registrados:\n")
print(dominios_unicos)

Hay 177 registros unicos registrados:

['api.wunderground.com' 'stork79.dropbox.com'
 'hpca-tier2.office.aol.com.ad.aol.aoltw.net'
 'safebrowsing.clients.google.com.home' 'fxfeeds.mozilla.com'
 'www.metasploit.com' 'aolmtcmxm03.office.aol.com'
 'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net'
 'aolmtcmxm02.office.aol.com' 'hpca-tier2.office.aol.com'
 'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net'
 'aolmtcmxm04.office.aol.com' 'safebrowsing.clients.google.com'
 'wpad.home' 'safebrowsing.clients.google.com.stayonline.net'
 'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net'
 'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com' 'AOLDTCMA04.office.aol.com'
 '192.168.22.110phpmyadmin' 'secure.informaction.com'
 'secure.informaction.com.localdomain'
 'safebrowsing.clients.google.com.localdomain' 'ueip.vmware.com'
 'en-us.fxfeeds.mozilla.com' '192.168.22.110phpmyadmin.localdomain'
 'time.windows.com' 'softwareupdate.vmware.com' 'proxim.ntkrnlpa.info'
 'portswigger.net' 'www.offensive-security.com'
 'www.offens

### ChatGPT Prompt para extraer el TLD de un dominio

Sure, here's the function to extract the Top-Level Domain (TLD) from a given domain:

```python
def get_tld(domain):
    """
    Function to extract the Top-Level Domain (TLD) from a given domain.

    Args:
    - domain (str): The domain from which the TLD is to be extracted.

    Returns:
    - tld (str): The TLD of the domain.
    """
    parts = domain.split('.')
    if len(parts) > 1:
        return '.'.join(parts[-2:])
    else:
        return domain

# Usage examples
domain_1 = "api.wunderground.com"
domain_2 = "safebrowsing.clients.google.com.home"

tld_1 = get_tld(domain_1)
tld_2 = get_tld(domain_2)

print("TLD of", domain_1, ":", tld_1)
print("TLD of", domain_2, ":", tld_2)
```

This function takes a domain as input and returns its TLD. It splits the domain into parts separated by dots (.), then joins the last two parts to get the TLD. If the domain has only one part (as in the second example "home"), it simply returns that value since there's no defined TLD in this case.

In [47]:
def get_tld(dominio):
    partes = dominio.split('.')
    if len(partes) > 1:
        return '.'.join(partes[-2:])
    else:
        return dominio

dominio_1 = "api.wunderground.com"
dominio_2 = "safebrowsing.clients.google.com.home"

tld_1 = get_tld(dominio_1)
tld_2 = get_tld(dominio_2)

print("TLD of", dominio_1, ":", tld_1)
print("TLD of", dominio_2, ":", tld_2)

TLD of api.wunderground.com : wunderground.com
TLD of safebrowsing.clients.google.com.home : com.home


In [48]:
# Crear la nueva columna 'domain_tld' con los TLD de cada dominio
df_dns_a['domain_tld'] = df_dns_a['rrname'].apply(get_tld)

# Eliminar todas las demás columnas excepto 'domain_tld'
df_dns_a = df_dns_a[['domain_tld']]

# Mostrar las primeras filas del DataFrame resultante para verificar
print(df_dns_a.head())

         domain_tld
0  wunderground.com
1       dropbox.com
2         aoltw.net
5  wunderground.com
6          com.home


C:\Users\oestr\AppData\Local\Temp\ipykernel_43664\2960914219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dns_a['domain_tld'] = df_dns_a['rrname'].apply(get_tld)
